In [332]:
# User visits problem from the rel-avito dataset
# on relbench: https://relbench.stanford.edu/datasets/rel-avito/#user-visits

# 1) https://demo.kurve.ai
# 2) create graph of rel-avito data
# 3) assign user as parent node with depth 4

In [355]:
import pandas as pd
from torch_frame.utils import infer_df_stype
import catboost
from sklearn import metrics

In [356]:
# train cut date of 5/8/15
train_path = 'https://kurve-customers.s3.amazonaws.com/4e1a245a-3065-4600-bb0e-a92e06ee835c/7/output/user_visits_train'


In [361]:
df = pd.read_parquet(train_path)
#df = pd.concat([pd.read_parquet(train_path), df])

In [362]:
df.shape

(47009, 314)

In [363]:
# re-execute compute graph with cut date of 5/14/15
test = pd.read_parquet(train_path)
test.shape

(64840, 311)

In [364]:
len(df.User_UserID.unique())

47009

In [365]:
[c for c in df.columns if 'label' in c]

['visit_ads_visited_label']

In [366]:
df[df['visit_ads_visited_label']>1].shape

(26943, 314)

In [367]:
df['label'] = df['visit_ads_visited_label'].apply(lambda x: 1 if x > 1 else 0)

In [368]:
df['label'].sum()/len(df)

0.5731455678699824

In [369]:
test['label'] = test['visit_ads_visited_label'].apply(lambda x: 1 if x > 1 else 0)

In [370]:
stypes = infer_df_stype(df)

In [371]:
len(stypes)

315

In [372]:
features = [
    k for k,v in stypes.items()
    if str(v) == 'numerical'
    and 'label' not in k
    and not k.startswith('User_')
]

In [373]:
features = [c for c in features if c in test.columns]# and c != 'visit_seconds_since_last']

In [374]:
target = 'label'

In [375]:
import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier, Pool

# -------------------------------------------------
# 1. Split off the *final* test set (once!)
# -------------------------------------------------
X_train_full, X_test, y_train_full, y_test = train_test_split(
    df[features], df[target],
    test_size=0.20,          # 20 % held-out test
    stratify=df[target],
    random_state=42
)

# -------------------------------------------------
# 2. K-Fold CV on the remaining 80 %
# -------------------------------------------------
k = 3                                   # change to 10, etc.
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

fold_aucs = []
test_preds = np.zeros(len(X_test))      # out-of-fold predictions on *test*
oof_preds = np.zeros(len(X_train_full)) # optional: OOF on training data

# CatBoost pools (optional but faster)
train_pool = Pool(X_train_full, y_train_full)  # , cat_features=cat_features)

for fold, (idx_tr, idx_va) in enumerate(skf.split(X_train_full, y_train_full), 1):
    print(f"\n=== Fold {fold} ===")
    
    X_tr, X_va = X_train_full.iloc[idx_tr], X_train_full.iloc[idx_va]
    y_tr, y_va = y_train_full.iloc[idx_tr], y_train_full.iloc[idx_va]

    # -----------------------------------------------------------------
    # 3. Fit on the *training* split of this fold
    # -----------------------------------------------------------------
    mdl = catboost.CatBoostClassifier(
        # objective & metrics
        loss_function="Logloss",
        eval_metric="AUC",
        custom_metric=["AUC", "PRAUC", "F1", "Recall", "Precision", "Logloss"],
        use_best_model=True,
    
        # capacity vs regularization
        iterations=2000,
        learning_rate=0.05,
        depth=5,
        l2_leaf_reg=10,
    
        # randomness / bagging / feature subsampling
        bootstrap_type="Bayesian",
        bagging_temperature=0.75,
        rsm=0.8,
        random_strength=0.5,
    
        # class imbalance handling
        auto_class_weights="Balanced",
        #class_weights=[0.6,0.4],
    
        # borders / leaves
        feature_border_type="GreedyLogSum",
        min_data_in_leaf=20,
        boosting_type="Plain",
    
        # early stopping
        od_type="Iter",
        od_wait=150,
    
        verbose=200
    )
    mdl.fit(
        X_tr, y_tr,
        eval_set=(X_va, y_va),
        use_best_model=False,
        verbose=False
    )

    # -------------------------------------------------
    # 4. Validation AUC for this fold
    # -------------------------------------------------
    val_pred = mdl.predict_proba(X_va)[:, 1]
    val_auc  = roc_auc_score(y_va, val_pred)
    fold_aucs.append(val_auc)
    print(f"Fold {fold} validation AUC : {val_auc:.4f}")

    # -------------------------------------------------
    # 5. Accumulate predictions on the *final* test set
    # -------------------------------------------------
    test_preds += mdl.predict_proba(X_test)[:, 1] / k

    # (optional) OOF on training data
    oof_preds[idx_va] = val_pred

# -------------------------------------------------
# 6. Final metrics
# -------------------------------------------------
print("\n=== CV Summary ===")
print(f"Mean CV AUC : {np.mean(fold_aucs):.4f} ± {np.std(fold_aucs):.4f}")
print(f"Folds AUC   : {[f'{a:.4f}' for a in fold_aucs]}")

test_auc = roc_auc_score(y_test, test_preds)
print(f"\nFinal test AUC (averaged over {k} folds): {test_auc:.4f}")

# -------------------------------------------------
# 7. (Optional) Refit on the *whole* train_full for deployment
# -------------------------------------------------
final_mdl = catboost.CatBoostClassifier(
        # objective & metrics
        loss_function="Logloss",
        eval_metric="AUC",
        custom_metric=["AUC", "PRAUC", "F1", "Recall", "Precision", "Logloss"],
        #use_best_model=True,
    
        # capacity vs regularization
        #iterations=2000,
        iterations=int(mdl.best_iteration_ * 1.1),  # a bit more than best
        learning_rate=0.05,
        depth=5,
        l2_leaf_reg=10,
    
        # randomness / bagging / feature subsampling
        bootstrap_type="Bayesian",
        bagging_temperature=0.75,
        rsm=0.8,
        random_strength=0.5,
    
        # class imbalance handling
        auto_class_weights="Balanced",
        #class_weights=[0.6,0.4],
    
        # borders / leaves
        feature_border_type="GreedyLogSum",
        min_data_in_leaf=20,
        boosting_type="Plain",
    
        # early stopping
        od_type="Iter",
        od_wait=150,
    
        verbose=200
    )
final_mdl.fit(X_train_full, y_train_full)


=== Fold 1 ===
Fold 1 validation AUC : 0.7812

=== Fold 2 ===
Fold 2 validation AUC : 0.7823

=== Fold 3 ===
Fold 3 validation AUC : 0.7816

=== CV Summary ===
Mean CV AUC : 0.7817 ± 0.0004
Folds AUC   : ['0.7812', '0.7823', '0.7816']

Final test AUC (averaged over 3 folds): 0.7771
0:	total: 7.49ms	remaining: 2.49s
200:	total: 1.9s	remaining: 1.25s
332:	total: 3.15s	remaining: 0us


In [376]:
test.shape

(64840, 312)

In [377]:
test['prob'] = final_mdl.predict_proba(test[features])[:,1]

In [378]:
metrics.roc_auc_score(test[target], test['prob'])

0.7080718146496892

In [379]:
list(reversed(sorted(zip(final_mdl.feature_names_, final_mdl.feature_importances_), key=lambda x: x[1])))

[('visit_seconds_since_last_view_max', 10.071429700255896),
 ('visit_seconds_since_last_view_sum', 8.991875430021052),
 ('visit_seconds_since_last', 5.993156417230671),
 ('visit_num_events_1d', 3.47870209671292),
 ('visit_1dv3_change', 3.315239233995453),
 ('subvisit_7dv14_change_sum', 3.0555726814244104),
 ('subvisit_seconds_since_last_max', 2.5798532753561156),
 ('visit_num_events_7d', 2.396816654072166),
 ('visit_seconds_since_last_view_avg', 1.9470977089521118),
 ('visit_num_events_3d', 1.9079441379568638),
 ('visit_num_events_730d', 1.6564224664745566),
 ('subvisit_90dv180_change_sum', 1.5355618366035604),
 ('visit_num_events_30d', 1.4591529553907638),
 ('visit_4dv7_change', 1.4262157172664554),
 ('visit_num_events_60d', 1.4036410222831115),
 ('visit_num_events_180d', 1.3372656770580569),
 ('visit_num_events_4d', 1.3024079709904524),
 ('AdsI_Price_avg', 1.242780642139085),
 ('sch_IsUserLoggedOn_avg', 1.2373251331305257),
 ('AdsI_Price_min', 1.2031202756442572),
 ('AdsI_Price_max',

In [256]:
test['label'].sum()/len(test)

0.41376131125149396

In [257]:
df['label'].sum()/len(df)

0.5695601825571627